# Wrangle the XAI Evaluation Results

# Imports

## Import Libraries

In [514]:
#Data wrangling
from imp import reload
import os
import numpy as np
import pandas as pd
import re
# Make sure joblib version is 1.2.0
import joblib
from scipy.stats import rankdata


# import scipy.signal


# #XAI Eval
# import ROAD
# import shap


from importlib import reload
import DataCleaningUtils
reload(DataCleaningUtils)
from DataCleaningUtils import *


## Import ROAD evaluation results

### Set paths of files to import

In [497]:
# Simulated Datasets
dir_path = '/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Model Explanation/ROAD Output'
# list to store files name
res_ = []
res = []
for (dir_path, dir_names, file_names) in os.walk(dir_path):
        res_.extend(file_names)
for file in res_:
    if file not in ['GroundTruth.csv', '.Rhistory', '.DS_Store']:
        res.append(file)

#Extracting the names of the datasets
res = [re.sub(r'csv.*', 'csv', i) for i in res]

# Eliminating repeated names
res = np.unique(res)
# Eliminate names that have a parenthesis
res = np.array([i for i in res if 'skew' not in i])
res = np.array([i for i in res if '30000' not in i])
res = np.array([i for i in res if '(' not in i])
print(res)

#Models of interest
model_names = np.array(['RandomForest','LogisticRegression','XGBoost', 'FauxModel'])



#Explainers of interest
xai_of_int = np.array(['ShapLinear', 'ShapTree', 'ShapPermutation', 'ShapSampling', 'ShapKernelKmeans', 'ShapExact', 'ShapFlow'])

#Number of Iterations
num_iter = 10
iter_list = list(range(num_iter))

import itertools
#Create a list of all possible combinations of models and explainers
comb = list(itertools.product(model_names, xai_of_int, res, iter_list))
len(comb)

['0_vars_corr_0HC_n100.csv' '0_vars_corr_0HC_n1000.csv'
 '0_vars_corr_0HC_n200.csv' '0_vars_corr_0HC_n300.csv'
 '0_vars_corr_0HC_n400.csv' '0_vars_corr_0HC_n500.csv'
 '0_vars_corr_0HC_n600.csv' '0_vars_corr_0HC_n700.csv'
 '0_vars_corr_0HC_n800.csv' '0_vars_corr_0HC_n900.csv'
 '1_vars_corr_1HC_n100.csv' '1_vars_corr_1HC_n1000.csv'
 '1_vars_corr_1HC_n300.csv' '1_vars_corr_1HC_n500.csv'
 '1_vars_corr_1HC_n700.csv' '1_vars_corr_1HC_n900.csv'
 '2_vars_corr_1HC_n100.csv' '2_vars_corr_1HC_n1000.csv'
 '2_vars_corr_1HC_n300.csv' '2_vars_corr_1HC_n500.csv'
 '2_vars_corr_1HC_n700.csv' '2_vars_corr_1HC_n900.csv'
 '2_vars_corr_2HC_n100.csv' '2_vars_corr_2HC_n1000.csv'
 '2_vars_corr_2HC_n300.csv' '2_vars_corr_2HC_n500.csv'
 '2_vars_corr_2HC_n700.csv' '2_vars_corr_2HC_n900.csv'
 '3_vars_corr_1HC_n100.csv' '3_vars_corr_1HC_n1000.csv'
 '3_vars_corr_1HC_n300.csv' '3_vars_corr_1HC_n500.csv'
 '3_vars_corr_1HC_n700.csv' '3_vars_corr_1HC_n900.csv'
 '3_vars_corr_2HC_n100.csv' '3_vars_corr_2HC_n1000.csv'
 '3_

12880

### Import Ground Truth

In [498]:
## Because we are using sinthetic data, we know how each variable is causally affecting the target.
# Below, we are going to rank in descending order (larger number means more important) the features.
ground = pd.read_csv('/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Synthetic Data/GroundTruth.csv')
ground.columns.values[0] = 'Features'
ground = ground.melt('Features', var_name='Dataset', value_name='Rank')
ground_ = ground.set_index(['Features']).groupby('Dataset').transform(rankdata).reset_index()
ground_['Dataset'] = ground['Dataset']
ground = ground_

# Ensure that for the Skew datasets, the rank is in the correct order
# ground.loc[np.logical_and(ground.Dataset.str.contains('skew'), np.logical_not(ground.Features.str.contains('Noise'))), ground.columns[1]] = 3
# ground.loc[np.logical_and(ground.Dataset.str.contains('skew'), ground.Features.str.contains('Noise')), ground.columns[1]]= 1


ground

,Features,Rank,Dataset
0,C1,4.0,2_vars_corr_1HC_n100.csv
1,C2,3.0,2_vars_corr_1HC_n100.csv
2,C3,2.0,2_vars_corr_1HC_n100.csv
3,Noise,1.0,2_vars_corr_1HC_n100.csv
4,C1,4.0,1_vars_corr_1HC_n100.csv
...,...,...,...
275,Noise,1.0,0_vars_corr_0HC_n900.csv
276,C1,4.0,0_vars_corr_0HC_n1000.csv
277,C2,3.0,0_vars_corr_0HC_n1000.csv
278,C3,2.0,0_vars_corr_0HC_n1000.csv


<b> Dimensions of the evaluation files </b> <br> <br>
list: 0 = top, 1 = bottom, 2 = random, 3 = shap values <br> <br>
Axis 0 = metric : <br>
<pre> Metrics: 0 = accu, 1 = accu_balanced, 2 = f1, 3 = auroc, 4 = auprc, 5 = recall </pre>
Axis 1 = stage of imputation <br>
Axis 2 = repeats

### Import the data keep only the Score, Similarity, and ROAD metrics (in case we want to plot later on)

In [515]:
# Clean up results of road
path = '/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Model Explanation/ROAD Output'
score, score_bottom, score_top, similarity, kendall, random, noise = clean_road_results(model_names, xai_of_int, res, iter_list, ground, path)


In [340]:
score['2_vars_corr_1HC_n1000.csv']['RandomForest'].keys()

dict_keys(['ShapTree', 'ShapPermutation', 'ShapSampling', 'ShapKernelKmeans', 'ShapExact', 'ShapFlow'])

In [341]:
score['3_vars_corr_2HC_n100B.csv']['XGBoost']['ShapTree'][3, :]

array([0.01132587, 0.01372088, 0.01346247, 0.01335631, 0.0137839 ,
       0.01215152, 0.0137839 , 0.01355071, 0.01355071, 0.01373505])

In [231]:
#Get the score and similarity for every file, model, and explainer
score = {}
score_bottom = {}
score_top = {}
similarity = {}
kendall = {}
random = {}


for file in res:

    score_ = {}
    score_bottom_ = {}
    score_top_ = {}
    similarity_ = {}
    kendall_ = {}
    distance_ = {}
    random_ = {}
    
    #Get the ground truth for a particular file
    groundnp = ground.loc[ground.Dataset == file].Rank.values
    for model in model_names:

        _score_ = {}
        _score_bottom_ = {}
        _score_top_ = {}
        _similarity_ = {}
        _kendall_ = {}
        _random_ = {}
        
        #Create a dictionary to store the results
        for xai in xai_of_int:

            for rep in iter_list:
                path = f'{dir_path}/{file}_{model}_{xai}_{rep}.joblib'
                
                if not os.path.isfile(path):
                    does_not_exist = True
                    # print(f'{path} does not exist')
                    continue
                
                
                top, bottom, rand, shap_values = joblib.load(path)

                if shap_values is None:
                    does_not_exist = True
                    continue

                does_not_exist = False

                #stack the results
                if rep == 0:
                    top_ = top
                    bottom_ = bottom
                    rand_ = rand
                    shap_values_ = shap_values
                else:
                    top_ = np.dstack([top_, top])
                    bottom_ = np.dstack([bottom_, bottom])
                    rand_ = np.dstack([rand_, rand])
                    shap_values_ = np.dstack([shap_values_, shap_values])
    
            
            if does_not_exist:
                continue

            evals = [top_, bottom_, rand_, shap_values_]
  


      
                
            #Get the score for a particular file, model, and explainer
            _score_[xai] = road_score_alt1(evals)
            _score_bottom_[xai] = get_score_bottom(evals)
            _score_top_[xai] = get_score_top(evals)
            _similarity_[xai] = similarity_score(evals)
            _kendall_[xai] = get_kendall(evals, groundnp)
            _random_[xai] = get_random(evals)
            
        #  #Plot the results and Save the figure
        #     road_plot(evals, model, xai, file)
        #     plt.savefig(f'/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/XAI method performacne when Explainaing the PORT Dataset/Figures/Simulated/Full Figure/ROAD/{file}_{model}_{xai}.pdf', dpi=600, bbox_inches = 'tight', transparent = False)
        #     plt.close()

        if does_not_exist:
            continue
        
        score_[model] = _score_
        score_bottom_[model] = _score_bottom_
        score_top_[model] = _score_top_
        similarity_[model] = _similarity_
        kendall_[model] = _kendall_
        random_[model] = _random_
    
    if does_not_exist:
        continue

    score[file] = score_
    score_bottom[file] = score_bottom_
    score_top[file] = score_top_
    similarity[file] = similarity_
    kendall[file] = kendall_
    random[file] = random_

### Save the cleaned up results

In [265]:
save_path = '/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Results/XAI Evaluation Data'
joblib.dump(score, f'{save_path}/score.joblib', compress=3)
joblib.dump(score_bottom, f'{save_path}/score_bottom.joblib', compress=3)
joblib.dump(score_top, f'{save_path}/score_top.joblib', compress=3)
joblib.dump(similarity, f'{save_path}/similarity.joblib', compress=3)
joblib.dump(kendall, f'{save_path}/kendall.joblib', compress=3)
joblib.dump(random, f'{save_path}/random.joblib', compress=3)
joblib.dump(noise, f'{save_path}/noise.joblib', compress=3)

['/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Results/XAI Evaluation Data/noise.joblib']

# Wrangle results

## Extract results into dataframes

In [538]:
df_score_mean, df_score_bottom_mean, df_score_top_mean, df_sim_std, df_kendall, df_random, df_noise = results_to_dataframes(score, score_bottom, score_top, similarity, kendall, random, noise, metric = 4)

In [234]:
## Make a Data Frame of ROAR Score
df_score_mean = {}
for file, models in score.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results[3]
        df_models[model] = df_exp
    df_score_mean[file] = pd.DataFrame(df_models).T
df_score_mean = pd.concat(df_score_mean)

## Make a Data Frame for ROAR Score Bottom
df_score_bottom_mean = {}
for file, models in score_bottom.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results[3]
        df_models[model] = df_exp
    df_score_bottom_mean[file] = pd.DataFrame(df_models).T
df_score_bottom_mean = pd.concat(df_score_bottom_mean)

## Make a Data Frame for ROAR Score Top
df_score_top_mean = {}
for file, models in score_top.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results[3]
        df_models[model] = df_exp
    df_score_top_mean[file] = pd.DataFrame(df_models).T
df_score_top_mean = pd.concat(df_score_top_mean)



#Data frame of the score
df_sim_std = {}
for file, models in similarity.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results
        df_models[model] = df_exp
    df_sim_std[file] = pd.DataFrame(df_models).T
df_sim_std = pd.concat(df_sim_std)


#Data frame of kendall's tau
df = {}
for file, models in kendall.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results[0]
        df_models[model] = df_exp
    df[file] = pd.DataFrame(df_models).T
df_kendall = pd.concat(df)


# #Data frame of distance
# df = {}
# for file, models in distance.items():
#     df_models = {}
#     for model, exp in models.items():
#         df_exp = {}
#         for explainer, roar_results in exp.items():
#             df_exp[explainer] = roar_results[0]
#         df_models[model] = df_exp
#     df[file] = pd.DataFrame(df_models).T
# df_distance = pd.concat(df)


#Data frame of random
df = {}
for file, models in random.items():
    df_models = {}
    for model, exp in models.items():
        df_exp = {}
        for explainer, roar_results in exp.items():
            df_exp[explainer] = roar_results[3]
        df_models[model] = df_exp
    df[file] = pd.DataFrame(df_models).T
df_random = pd.concat(df)

## Join everything into a single dataframe

In [539]:
def tidying(df, value = None):
    df = df.reset_index().melt(['level_0', 'level_1', 'level_2'])
    df.columns = ['Dataset', 'Explainer', 'Repeat', 'Model', f'{value}']
    return df.set_index(['Dataset', 'Explainer', 'Repeat', 'Model'])

df = pd.concat(
    [
        tidying(df_score_mean, 'score'), 
        tidying(df_score_bottom_mean, 'score_bottom'), 
        tidying(df_score_top_mean, 'score_top'), 
        tidying(df_kendall, 'kendall'), 
        tidying(df_random, 'random'),
        tidying(df_noise, 'noise')
    ], 
    axis=1)
df = df.iloc[:, ~df.columns.duplicated()].dropna(axis=0, thresh=5).reset_index()
df

,Dataset,Explainer,Repeat,Model,score,score_bottom,score_top,kendall,random,noise
0,0_vars_corr_0HC_n100.csv,ShapExact,0,RandomForest,2.610254,0.272746,-0.009424,0.333333,0.646414,0.01592
1,0_vars_corr_0HC_n100.csv,ShapExact,1,RandomForest,2.610254,0.184850,0.099341,0.333333,0.724540,0.01592
2,0_vars_corr_0HC_n100.csv,ShapExact,2,RandomForest,2.610254,0.078741,0.203077,0.333333,0.818854,0.01592
3,0_vars_corr_0HC_n100.csv,ShapExact,3,RandomForest,2.610254,0.069477,0.211011,0.333333,0.827088,0.01592
4,0_vars_corr_0HC_n100.csv,ShapExact,4,RandomForest,2.610254,0.098595,0.185524,0.333333,0.801206,0.01592
...,...,...,...,...,...,...,...,...,...,...
10575,3_vars_corr_2HC_n900B.csv,ShapSampling,5,FauxModel,2.625683,0.000000,0.217564,0.333333,0.838945,0.00000
10576,3_vars_corr_2HC_n900B.csv,ShapSampling,6,FauxModel,2.625683,0.152300,0.076989,0.333333,0.711173,0.00000
10577,3_vars_corr_2HC_n900B.csv,ShapSampling,7,FauxModel,2.625683,0.000000,0.217564,0.333333,0.838945,0.00000
10578,3_vars_corr_2HC_n900B.csv,ShapSampling,8,FauxModel,2.625683,0.185601,0.039247,0.333333,0.683236,0.00000


## Add known information about the datasets to the results

### Extract the Class Imbalance (Skew) of each Dataset

In [526]:
l = []
for file in res:
    dir_path = f'/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Synthetic Data/Data files with summed target/{file}'
    l.append(pd.read_csv(dir_path).Target.mean())
    # print(pd.read_csv(dir_path).Target.mean())

### Extract the pattern of confounding
This is only relevant to distinguish the case where we have 2 Hidden Confounders and all 3 predictive variables being confounded.

In [527]:
a = [re.sub(r".*_n|\.csv|\d*|_.*", '', x) for x in res]
f = lambda x: 'A' if x == '' else x
a = [f(x) for x in a]

### Join the known information into the results

In [540]:
#Put in some known information about the datasets
df = df.merge(pd.DataFrame(
    {'Dataset' : res,
    'Skew': l,
    'Samples': [np.int64(re.sub(r".*_n|_skew.*|\.csv|B", '', x)) for x in res],
    'HC': [np.int64(re.sub(r".*_corr_|HC.*", '', x)) for x in res],
    'ConfVars': [np.int64(re.sub(r"_.*", '', x)) for x in res],
    'AB': a}), on='Dataset').set_index('Dataset')

print(df.shape)
df.head()

(10580, 14)


,Explainer,Repeat,Model,score,score_bottom,score_top,kendall,random,noise,Skew,Samples,HC,ConfVars,AB
Dataset,,,,,,,,,,,,,,
0_vars_corr_0HC_n100.csv,ShapExact,0,RandomForest,2.610254,0.272746,-0.009424,0.333333,0.646414,0.01592,0.5,100,0,0,A
0_vars_corr_0HC_n100.csv,ShapExact,1,RandomForest,2.610254,0.184850,0.099341,0.333333,0.724540,0.01592,0.5,100,0,0,A
0_vars_corr_0HC_n100.csv,ShapExact,2,RandomForest,2.610254,0.078741,0.203077,0.333333,0.818854,0.01592,0.5,100,0,0,A
0_vars_corr_0HC_n100.csv,ShapExact,3,RandomForest,2.610254,0.069477,0.211011,0.333333,0.827088,0.01592,0.5,100,0,0,A
0_vars_corr_0HC_n100.csv,ShapExact,4,RandomForest,2.610254,0.098595,0.185524,0.333333,0.801206,0.01592,0.5,100,0,0,A


In [550]:
#Average the results across the iterations
df = df.groupby(['Explainer', 'Model', 'Dataset', 'AB']).mean().drop('Repeat', axis=1)

# Save the Results as a single CSV for easy access

In [553]:
df.to_csv('/Users/eddie/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Research/Projects/Explainability Method Comparison/Data-ML-XAI-Eval/Results/XAI Evaluation Data/CleanEvals.csv')